In [5]:
# load  subject nifti
for sj in np.hstack([best_subjects_roisum,181,182,183]):
    data=nb.load(os.path.join(out_dir,best_subjects,'prfresults_subject_%d_ecc_r2_mask.nii'%sj)).get_data()

    pl.figure(figsize=(15,5))
    pl.title(sj)
    data[data==0] = np.nan
    pl.subplot(131)
    pl.imshow(data[52,25:50,5:20,0].T,origin='lowerleft',cmap='hsv',clim=[0,360],interpolation='nearest')
    sn.despine(offset=10)
    pl.subplot(132)
    pl.imshow(data[30:60,38,5:20,0].T,origin='lowerleft',cmap='hsv',clim=[0,360],interpolation='nearest')
    sn.despine(offset=10)
    pl.subplot(133)
    pl.imshow(data[20:70,30:50,11,0].T,origin='lowerleft',cmap='hsv',clim=[0,360],interpolation='nearest')
    sn.despine(offset=10)

    pl.savefig()


NameError: name 'best_subjects_roisum' is not defined

In [ ]:
# create individual subject nifti mask

# mask 1: as in paper

Let's first create a mask as they did in the paper, this is based on the boundary between a 'noise' and 'signal pool within the r-squareds across all grayordinates. To do this, we fit a a gaussian mixture model and define the boundary between the pools as the transition point between the gaussians:

In [25]:
# now for a mask one
for best_subjects in ['best_subjects_v123','best_subjects_cer','best_subjects_cergmm']:
    print 'now masking for %s'%best_subjects

    for sj in np.hstack([eval(best_subjects),181,182,183]):#allresults.shape[1]-1]:#range(allresults.shape[1]-1,allresults.shape[1]):

        # get results for this subject
        these_results = copy.copy(allresults[0,sj])

        r2s = these_results[4]

        gmm = GMM(n_components = 2)
        gmm = gmm.fit(np.expand_dims(r2s,1))

        x = np.linspace(0,100,10000)
        logprob, responsibilities = gmm.score_samples(np.expand_dims(x,1))

        pdf = np.exp(logprob)
        pdf_individual = responsibilities * pdf[:, np.newaxis]

        thresh = x[np.where(pdf_individual[:,0]>pdf_individual[:,1])[0]][0]
        if thresh == 0:
            thresh = x[np.where(pdf_individual[:,1]>pdf_individual[:,0])[0]][0]

        print 'subject %d, r2 threshold: %.2f'%(sj,thresh)

        # define mask with this threshold
        mask = (these_results[4]<thresh)

        these_results[:,mask]= np.nan

        # save results
        cii = os.path.join(out_dir,best_subjects,'prfresults_subject_%d_mask_paper.dscalar.nii'%sj)
        cifti.write(cii, these_results, (cifti.Scalar.from_names(['ang', 'ecc', 'gain', 'meanvol', 'R2', 'rfsize']), bm))
        # and convert the created cifti file to gii files
        wbc_c = wbc.format(cii=cii, cii_n=cii[:-4])
        os.system(wbc_c)    



subject 4, r2 threshold: 2.21
subject 116, r2 threshold: 2.48
subject 7, r2 threshold: 2.87
subject 171, r2 threshold: 2.49
subject 58, r2 threshold: 2.69
subject 74, r2 threshold: 2.49
subject 178, r2 threshold: 2.79
subject 34, r2 threshold: 2.63
subject 32, r2 threshold: 2.18
subject 27, r2 threshold: 3.89
subject 181, r2 threshold: 8.08
subject 182, r2 threshold: 8.19
subject 183, r2 threshold: 10.51
subject 7, r2 threshold: 2.87
subject 97, r2 threshold: 2.57
subject 169, r2 threshold: 3.17
subject 47, r2 threshold: 4.04
subject 101, r2 threshold: 2.78
subject 27, r2 threshold: 3.89
subject 45, r2 threshold: 3.24
subject 25, r2 threshold: 2.74
subject 156, r2 threshold: 2.83
subject 141, r2 threshold: 2.64
subject 181, r2 threshold: 8.08
subject 182, r2 threshold: 8.19
subject 183, r2 threshold: 10.51
subject 47, r2 threshold: 4.04
subject 7, r2 threshold: 2.87
subject 97, r2 threshold: 2.57
subject 169, r2 threshold: 3.17
subject 101, r2 threshold: 2.78
subject 45, r2 threshold: 

In [26]:
rsync -a --no-g --no-p -vzhe ssh --progress vanes@aeneas.labs.vu.nl:/home/shared/2018/visual/hcp_cerebellum/best_subjects_cer /Users/daanvanes/Downloads/hcp_cerebellum

SyntaxError: invalid syntax (<ipython-input-26-1628e59d24dd>, line 1)

# mask 2: GMM on cerebellum only

Perhaps the signal and noise pools between the cortex and cerebellum are different. Signal is usually lower in the cerebellum, so perhaps the R2 threshold derived from the cortex is too high. 

In [28]:
# load cerebellum_mask from the cerebellum atlas
fncer = '/home/vanes/bin/fsl/data/atlases/Cerebellum/Cerebellum-MNIfnirt-maxprob-thr50-2mm.nii.gz'
cerimg = nb.load(fncer)
cdata = cerimg.get_data()
cmask = (cdata!=0)

for best_subjects in ['best_subjects_v123','best_subjects_cer','best_subjects_cergmm']:
    print 'now masking for %s'%best_subjects

    for sj in np.hstack([eval(best_subjects),181,182,183]):#allresults.shape[1]-1]:#range(allresults.shape[1]-1,allresults.shape[1]):

        # load the prf results nifti
        fn = os.path.join(out_dir,'all_subjects','prfresults_subject_%d.dscalar_data_sub.nii'%sj)
        img = nb.load(fn)
        data = img.get_data()

        # mask r2s with cerebellum mask:
        r2s = np.ravel(data[cmask,4])

        # fit gaussian mixture model to define r2 threshold
        gmm = GMM(n_components = 2)
        gmm = gmm.fit(np.expand_dims(r2s,1))

        x = np.linspace(0,100,10000)
        logprob, responsibilities = gmm.score_samples(np.expand_dims(x,1))

        pdf = np.exp(logprob)
        pdf_individual = responsibilities * pdf[:, np.newaxis]

        thresh = x[np.where(pdf_individual[:,0]>pdf_individual[:,1])[0]][0]
        if thresh == 0:
            thresh = x[np.where(pdf_individual[:,1]>pdf_individual[:,0])[0]][0]

        print 'subject %d, r2 threshold: %.2f'%(sj,thresh)

        # define mask with this threshold
        mask = (data[:,:,:,4]<thresh)

        #mask data
        data[mask] = np.nan

        # save data
        out_fn = os.path.join(out_dir,best_subjects,'prfresults_subject_%d_mask_cergmm.nii'%sj)        
        new_data = nb.Nifti1Image(data,affine=img.affine,header=img.header)
        nb.save(new_data,out_fn[:-4])
        
    

now masking for best_subjects_v123
subject 4, r2 threshold: 1.57
subject 116, r2 threshold: 1.55
subject 7, r2 threshold: 2.09
subject 171, r2 threshold: 1.53
subject 58, r2 threshold: 1.53
subject 74, r2 threshold: 1.43
subject 178, r2 threshold: 1.57
subject 34, r2 threshold: 1.18
subject 32, r2 threshold: 1.63
subject 27, r2 threshold: 1.86
subject 181, r2 threshold: 2.92
subject 182, r2 threshold: 3.32
subject 183, r2 threshold: 3.93
now masking for best_subjects_cer
subject 7, r2 threshold: 2.09
subject 97, r2 threshold: 2.10
subject 169, r2 threshold: 1.84
subject 47, r2 threshold: 1.96
subject 101, r2 threshold: 2.04
subject 27, r2 threshold: 1.86
subject 45, r2 threshold: 1.96
subject 25, r2 threshold: 1.72
subject 156, r2 threshold: 1.70
subject 141, r2 threshold: 1.59
subject 181, r2 threshold: 2.92
subject 182, r2 threshold: 3.32
subject 183, r2 threshold: 3.93
now masking for best_subjects_cergmm
subject 47, r2 threshold: 1.96
subject 7, r2 threshold: 2.09
subject 97, r2 th

### observation from inspecting resulting niftis: 

r2 definition based on cerebellum only is too lenient, too many voxels survive
r2 definition based on whole brain is nice, but there's a clear distinction between an area with voxels of smaller than or larger than 0.5.

So make maps with ecc threshold and r2 threshold. First, let's make it easy by using thresholds from the paper:
2.2 for subjects and 9.8 for avg subjects for explained variance, and let's use the 0.5 eccentricity for all subjects.

# mask 1: as in paper

Let's first create a mask as they did in the paper, this is based on the boundary between a 'noise' and 'signal pool within the r-squareds across all grayordinates. To do this, we fit a a gaussian mixture model and define the boundary between the pools as the transition point between the gaussians:

In [25]:
# now for a mask one
for best_subjects in ['best_subjects_v123','best_subjects_cer','best_subjects_cergmm']:
    print 'now masking for %s'%best_subjects

    for sj in np.hstack([eval(best_subjects),181,182,183]):#allresults.shape[1]-1]:#range(allresults.shape[1]-1,allresults.shape[1]):

        # get results for this subject
        these_results = copy.copy(allresults[0,sj])

        r2s = these_results[4]

        gmm = GMM(n_components = 2)
        gmm = gmm.fit(np.expand_dims(r2s,1))

        x = np.linspace(0,100,10000)
        logprob, responsibilities = gmm.score_samples(np.expand_dims(x,1))

        pdf = np.exp(logprob)
        pdf_individual = responsibilities * pdf[:, np.newaxis]

        thresh = x[np.where(pdf_individual[:,0]>pdf_individual[:,1])[0]][0]
        if thresh == 0:
            thresh = x[np.where(pdf_individual[:,1]>pdf_individual[:,0])[0]][0]

        print 'subject %d, r2 threshold: %.2f'%(sj,thresh)

        # define mask with this threshold
        mask = (these_results[4]<thresh)

        these_results[:,mask]= np.nan

        # save results
        cii = os.path.join(out_dir,best_subjects,'prfresults_subject_%d_mask_paper.dscalar.nii'%sj)
        cifti.write(cii, these_results, (cifti.Scalar.from_names(['ang', 'ecc', 'gain', 'meanvol', 'R2', 'rfsize']), bm))
        # and convert the created cifti file to gii files
        wbc_c = wbc.format(cii=cii, cii_n=cii[:-4])
        os.system(wbc_c)    



subject 4, r2 threshold: 2.21
subject 116, r2 threshold: 2.48
subject 7, r2 threshold: 2.87
subject 171, r2 threshold: 2.49
subject 58, r2 threshold: 2.69
subject 74, r2 threshold: 2.49
subject 178, r2 threshold: 2.79
subject 34, r2 threshold: 2.63
subject 32, r2 threshold: 2.18
subject 27, r2 threshold: 3.89
subject 181, r2 threshold: 8.08
subject 182, r2 threshold: 8.19
subject 183, r2 threshold: 10.51
subject 7, r2 threshold: 2.87
subject 97, r2 threshold: 2.57
subject 169, r2 threshold: 3.17
subject 47, r2 threshold: 4.04
subject 101, r2 threshold: 2.78
subject 27, r2 threshold: 3.89
subject 45, r2 threshold: 3.24
subject 25, r2 threshold: 2.74
subject 156, r2 threshold: 2.83
subject 141, r2 threshold: 2.64
subject 181, r2 threshold: 8.08
subject 182, r2 threshold: 8.19
subject 183, r2 threshold: 10.51
subject 47, r2 threshold: 4.04
subject 7, r2 threshold: 2.87
subject 97, r2 threshold: 2.57
subject 169, r2 threshold: 3.17
subject 101, r2 threshold: 2.78
subject 45, r2 threshold: 

In [26]:
rsync -a --no-g --no-p -vzhe ssh --progress vanes@aeneas.labs.vu.nl:/home/shared/2018/visual/hcp_cerebellum/best_subjects_cer /Users/daanvanes/Downloads/hcp_cerebellum

SyntaxError: invalid syntax (<ipython-input-26-1628e59d24dd>, line 1)

# mask 2: GMM on cerebellum only

Perhaps the signal and noise pools between the cortex and cerebellum are different. Signal is usually lower in the cerebellum, so perhaps the R2 threshold derived from the cortex is too high. 

In [28]:
# load cerebellum_mask from the cerebellum atlas
fncer = '/home/vanes/bin/fsl/data/atlases/Cerebellum/Cerebellum-MNIfnirt-maxprob-thr50-2mm.nii.gz'
cerimg = nb.load(fncer)
cdata = cerimg.get_data()
cmask = (cdata!=0)

for best_subjects in ['best_subjects_v123','best_subjects_cer','best_subjects_cergmm']:
    print 'now masking for %s'%best_subjects

    for sj in np.hstack([eval(best_subjects),181,182,183]):#allresults.shape[1]-1]:#range(allresults.shape[1]-1,allresults.shape[1]):

        # load the prf results nifti
        fn = os.path.join(out_dir,'all_subjects','prfresults_subject_%d.dscalar_data_sub.nii'%sj)
        img = nb.load(fn)
        data = img.get_data()

        # mask r2s with cerebellum mask:
        r2s = np.ravel(data[cmask,4])

        # fit gaussian mixture model to define r2 threshold
        gmm = GMM(n_components = 2)
        gmm = gmm.fit(np.expand_dims(r2s,1))

        x = np.linspace(0,100,10000)
        logprob, responsibilities = gmm.score_samples(np.expand_dims(x,1))

        pdf = np.exp(logprob)
        pdf_individual = responsibilities * pdf[:, np.newaxis]

        thresh = x[np.where(pdf_individual[:,0]>pdf_individual[:,1])[0]][0]
        if thresh == 0:
            thresh = x[np.where(pdf_individual[:,1]>pdf_individual[:,0])[0]][0]

        print 'subject %d, r2 threshold: %.2f'%(sj,thresh)

        # define mask with this threshold
        mask = (data[:,:,:,4]<thresh)

        #mask data
        data[mask] = np.nan

        # save data
        out_fn = os.path.join(out_dir,best_subjects,'prfresults_subject_%d_mask_cergmm.nii'%sj)        
        new_data = nb.Nifti1Image(data,affine=img.affine,header=img.header)
        nb.save(new_data,out_fn[:-4])
        
    

now masking for best_subjects_v123
subject 4, r2 threshold: 1.57
subject 116, r2 threshold: 1.55
subject 7, r2 threshold: 2.09
subject 171, r2 threshold: 1.53
subject 58, r2 threshold: 1.53
subject 74, r2 threshold: 1.43
subject 178, r2 threshold: 1.57
subject 34, r2 threshold: 1.18
subject 32, r2 threshold: 1.63
subject 27, r2 threshold: 1.86
subject 181, r2 threshold: 2.92
subject 182, r2 threshold: 3.32
subject 183, r2 threshold: 3.93
now masking for best_subjects_cer
subject 7, r2 threshold: 2.09
subject 97, r2 threshold: 2.10
subject 169, r2 threshold: 1.84
subject 47, r2 threshold: 1.96
subject 101, r2 threshold: 2.04
subject 27, r2 threshold: 1.86
subject 45, r2 threshold: 1.96
subject 25, r2 threshold: 1.72
subject 156, r2 threshold: 1.70
subject 141, r2 threshold: 1.59
subject 181, r2 threshold: 2.92
subject 182, r2 threshold: 3.32
subject 183, r2 threshold: 3.93
now masking for best_subjects_cergmm
subject 47, r2 threshold: 1.96
subject 7, r2 threshold: 2.09
subject 97, r2 th

### observation from inspecting resulting niftis: 

r2 definition based on cerebellum only is too lenient, too many voxels survive
r2 definition based on whole brain is nice, but there's a clear distinction between an area with voxels of smaller than or larger than 0.5.

So make maps with ecc threshold and r2 threshold. First, let's make it easy by using thresholds from the paper:
2.2 for subjects and 9.8 for avg subjects for explained variance, and let's use the 0.5 eccentricity for all subjects.